In [21]:
# Import files distance_matrix_1.csv and distance_matrix_2.csv
# Voor foto: ![title](assumptions.png) in markdown cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import * 
import math 
import time
import seaborn as sns


# small_instance = False               # kleine dataset
# dataset = 1                         # kies 1 of 2

# # Import the data with delimeter as ; in stad of comma

# if dataset == 1:
#     distances = pd.read_excel('data.xlsx', sheet_name='Distance 1', index_col=0)
#     demand = pd.read_excel('data.xlsx', sheet_name='Demand 1')

# if dataset == 2:
#     distances = pd.read_excel('data.xlsx', sheet_name='Distance 2', index_col=0)
#     demand = pd.read_excel('data.xlsx', sheet_name='Demand 2')

# elif small_instance:
#     distances = pd.read_excel('data.xlsx', sheet_name='Distance_small', index_col=0)
#     demand = pd.read_excel('data.xlsx', sheet_name='Demand_small')
    

# # # Print the first 5 rows of the dataframe
# # print(distances.head())
# # print(demand.head())

# nodes = list(distances.columns) # first and last nodes are both the depot (0 and 14, for start and end point of the routes
# for i in nodes:
#     distances[i][i] = 99999   # penalise distance from node to itself so the model does not use these edges
# distances[0][nodes[-1]] = 99999      # penalise distance from depot to itself
# distances[nodes[-1]][0] = 99999     # penalise distance from depot to itself

# # Single Depot - Remove last row and column of distance data:
# distances = distances.iloc[:-1]
# distances = distances.iloc[:, :-1]
# demand = demand.iloc[:-1]


# # After loading the data
# distances.columns = distances.index


In [25]:
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix

# Function to generate random test dataset
def generate_test_dataset(num_nodes):
    np.random.seed(0)  # For reproducibility
    nodes = np.random.uniform(0, 100, size=(num_nodes, 2))  # Generate random coordinates
    delivery_demand = np.random.uniform(0, 100, size=(num_nodes,))  # Generate random delivery demands
    pickup_demand = (0.5 + np.random.uniform(0, 1, size=(num_nodes,))) * delivery_demand  # Compute pickup demand
    # Round delivery and pickup demands to numbers divisible by 5
    delivery_demand = np.round(delivery_demand / 5) * 5
    pickup_demand = np.round(pickup_demand / 5) * 5
    return pd.DataFrame({'X': nodes[:, 0], 'Y': nodes[:, 1], 'Delivery': delivery_demand, 'Pick-up': pickup_demand, 'Node': range(num_nodes)})

# Function to compute distance matrix
def compute_distance_matrix(df):
    coordinates = df[['X', 'Y']].values
    distances = distance_matrix(coordinates, coordinates)
    # Round values to numbers divisible by 5
    distances = np.round(distances / 5) * 5
    distance_df = pd.DataFrame(distances, index=df['Node'], columns=df['Node'])
    # Set column and index names to None
    distance_df.columns.name = None
    distance_df.index.name = None
    return distance_df

# Generate random test datasets with 7 to 15 nodes
datasets = {num_nodes: generate_test_dataset(num_nodes) for num_nodes in range(7, 16)}

# Compute distance matrices and create demand dataframes for each dataset
distance_matrices = {}
demand_dataframes = {}
for num_nodes, dataset in datasets.items():
    # Compute distance matrix
    distance_matrices[num_nodes] = compute_distance_matrix(dataset)
    
    # Create demand dataframe
    demand_dataframes[num_nodes] = dataset[['Node', 'Delivery', 'Pick-up']]


In [26]:
n = 15
distances = distance_matrices[n]  # get distance df with n nodes
demand = demand_dataframes[n]      # get demand df with n nodes
nodes = list(distances.columns) # first and last nodes are both the depot (0 and 14, for start and end point of the routes
for i in nodes:
    distances[i][i] = 99999   # penalise distance from node to itself so the model does not use these edges

distances

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,99999.0,20.0,15.0,20.0,55.0,30.0,20.0,80.0,55.0,30.0,45.0,10.0,45.0,45.0,30.0
1,20.0,99999.0,20.0,40.0,40.0,20.0,40.0,70.0,65.0,35.0,45.0,25.0,50.0,60.0,15.0
2,15.0,20.0,99999.0,25.0,60.0,40.0,30.0,65.0,45.0,40.0,60.0,15.0,30.0,40.0,25.0
3,20.0,40.0,25.0,99999.0,75.0,50.0,15.0,90.0,40.0,35.0,55.0,10.0,40.0,30.0,50.0
4,55.0,40.0,60.0,75.0,99999.0,25.0,65.0,95.0,105.0,50.0,40.0,65.0,90.0,100.0,45.0
5,30.0,20.0,40.0,50.0,25.0,99999.0,45.0,85.0,85.0,35.0,35.0,40.0,70.0,75.0,30.0
6,20.0,40.0,30.0,15.0,65.0,45.0,99999.0,95.0,55.0,20.0,45.0,20.0,55.0,45.0,50.0
7,80.0,70.0,65.0,90.0,95.0,85.0,95.0,99999.0,75.0,105.0,115.0,80.0,55.0,85.0,55.0
8,55.0,65.0,45.0,40.0,105.0,85.0,55.0,75.0,99999.0,75.0,95.0,45.0,20.0,15.0,65.0
9,30.0,35.0,40.0,35.0,50.0,35.0,20.0,105.0,75.0,99999.0,20.0,35.0,70.0,65.0,50.0


In [24]:
# write distance and demand files to csv:
for n in range(7, 16):
    distances = distance_matrices[n]  # get distance df with n nodes
    demand = demand_dataframes[n]      # get demand df with n nodes
    nodes = list(distances.columns) # first and last nodes are both the depot (0 and 14, for start and end point of the routes
    for i in nodes:
        distances[i][i] = 99999   # penalise distance from node to itself so the model does not use these edges


    distances.to_csv(f'distances_{n}.csv')
    demand.to_csv(f'demand_{n}.csv')

,Node,Delivery,Pick-up
0,0,25.0,30.0
1,1,75.0,55.0
2,2,45.0,30.0
3,3,55.0,45.0
4,4,0.0,0.0
5,5,60.0,65.0
6,6,60.0,55.0
7,7,60.0,90.0
8,8,95.0,55.0
9,9,70.0,50.0


In [3]:
Q = 180
v_lower = max(demand["Delivery"].sum() / Q , demand['Pick-up'].sum() / Q) 
v_lower.round()

2.0

# Sets and Parameters

In [92]:
# differ fleet size and capacity with sensitivity analysis 
n_customer_nodes = distances.shape[0] - 1

# lower bound for n_vehicles
v_lower = max(demand["Delivery"].sum() / Q , demand['Pick-up'].sum() / Q) 
n_vehicles  = v_lower.round()                   # Fleet size
n_vehicles = 2                                   # Fleet size
Q = 145                                         # Vehicle capacity, Homogeneous fleet
TL = 500                                      # Max route length for vehicle


# Create Node and Edge classes 
class Node():

    def __init__(self, nr, pickup_demand, delivery_demand):
        self.nr = nr
        self.pickup_demand = pickup_demand
        self.delivery_demand = delivery_demand

class Edge():

    def __init__(self, origin, destination, length):
        self.origin = origin
        self.destination = destination       
        self.length = length        # (or cost)
        
class Vehicle():

    def __init__(self, id, Q, TL):
        self.id = id
        self.Q = Q              # Capacity
        self.TL = TL            # Maximum route length 

# Generate Node and Edge objects from provided benchmark data
        
# Customer nodes N 
N = []
for index, row in demand.iterrows():
    if index in range(1, n_customer_nodes+1):
        node = Node(index, row['Pick-up'], row['Delivery'])
        N.append(node)

# T = [N ∪ {0}], all customer nodes in graph plus depot as start (0), 
T = []
for index, row in demand.iterrows():
    node = Node(index, row['Pick-up'], row['Delivery'])
    T.append(node)


edges = []
for i in T:
    for j in T:
        origin = i.nr 
        destination = j.nr
        length = distances[i.nr][j.nr]
        edges.append(Edge(origin, destination, length))
 
V = []
for v in range(1, n_vehicles+1):
    V.append(Vehicle(v, Q, TL))

print(f'\nSet N: \n')
for i in N:
    print(f'Node nr: {i.nr}, Pickup Demand: {i.pickup_demand}, Delivery Demand: {i.delivery_demand}')

print(f'\nSet T: \n')
for i in T:
    print(f'Node nr: {i.nr}, Pickup Demand: {i.pickup_demand}, Delivery Demand: {i.delivery_demand}')

print(f'\nSet V: \n')
for v in V:
    print(f' Vehicle {v.id}: Capacity {v.Q}, Max route length {v.TL}')


Set N: 

Node nr: 1, Pickup Demand: 30, Delivery Demand: 20
Node nr: 2, Pickup Demand: 30, Delivery Demand: 25
Node nr: 3, Pickup Demand: 15, Delivery Demand: 15
Node nr: 4, Pickup Demand: 30, Delivery Demand: 40
Node nr: 5, Pickup Demand: 15, Delivery Demand: 20
Node nr: 6, Pickup Demand: 15, Delivery Demand: 10
Node nr: 7, Pickup Demand: 20, Delivery Demand: 30
Node nr: 8, Pickup Demand: 35, Delivery Demand: 30
Node nr: 9, Pickup Demand: 10, Delivery Demand: 25
Node nr: 10, Pickup Demand: 15, Delivery Demand: 20
Node nr: 11, Pickup Demand: 15, Delivery Demand: 15
Node nr: 12, Pickup Demand: 30, Delivery Demand: 20
Node nr: 13, Pickup Demand: 30, Delivery Demand: 20

Set T: 

Node nr: 0, Pickup Demand: 0, Delivery Demand: 0
Node nr: 1, Pickup Demand: 30, Delivery Demand: 20
Node nr: 2, Pickup Demand: 30, Delivery Demand: 25
Node nr: 3, Pickup Demand: 15, Delivery Demand: 15
Node nr: 4, Pickup Demand: 30, Delivery Demand: 40
Node nr: 5, Pickup Demand: 15, Delivery Demand: 20
Node nr: 

# Variables

In [93]:
%%time
m = Model('VRPSPD')
##################### Decision Variables ########################
X = {}          # X_ijv = 1 if vehicle v travels from i to j, 0 otherwise
D = {}          # D_iv = The load remaining to be delivered by vehicle v when departing from node i - non-negative integer 
P = {}          # P_iv = The cumulative load picked by vehicle v when departing from node i, - non-negative integer 

# X
for i in T:
    for j in T:
        for v in V:
            y_ij = distances[i.nr][j.nr] 
            X[i.nr, j.nr, v.id] = m.addVar(obj = y_ij, lb=0,
                                vtype = GRB.BINARY, name = f'X_{i.nr},{j.nr},{v.id}')
            

# D & P
# print('\n P_iv and D_iv: \n')
for i in T:
    for v in V:
        # D
        D[i.nr, v.id] = m.addVar(obj = 0, lb=0,
                        vtype=GRB.INTEGER, name = f'D_{i.nr},{v.id}')
        
        # P
        P[i.nr, v.id] = m.addVar(obj = 0, lb=0,
                        vtype=GRB.INTEGER, name = f'P_{i.nr},{v.id}')

m.update()
m.setObjective(m.getObjective(), GRB.MINIMIZE)

CPU times: user 13.4 ms, sys: 1.34 ms, total: 14.7 ms
Wall time: 15.3 ms


# Constraints

In [94]:
%%time
#################################################### Constraints ###########################################

#################################################### C1 ####################################################
# Stipulates that each customer node must be visited by exactly once.
for j in N:  
    m.addConstr(quicksum(quicksum( X[i.nr, j.nr, v.id] for i in T) for v in V), 
                GRB.EQUAL, 1, name=f'C1_{j.nr}')



#################################################### C2 ####################################################
#  Ensures that each vehicle is used, exactly once. And that the vehicles leave and enter the depot once 
for v in V:
    m.addConstr( quicksum(X[0, j.nr, v.id] for j in N), GRB.EQUAL, 1, name='C2')
    m.addConstr( quicksum(X[j.nr, 0, v.id] for j in N), GRB.EQUAL, 1, name='C2*')



#################################################### C3 ####################################################
# Ensures that same vehicle arrives and departs from each node it serves.
for j in T: 
    for v in V:
        # C3
        m.addConstr(quicksum( X[i.nr, j.nr, v.id] for i in T)  
                    - quicksum( X[j.nr, i.nr, v.id] for i in T), GRB.EQUAL, 0, name=f'C3_{j.nr}_{v.id}') 



#################################################### C4 ###################################################
# C4
for i in T: 
    for v in V:

        # Ensures that the load on vehicle v, when departing from node i, is always lower than the vehicle capacity.
        m.addConstr( D[i.nr, v.id] + P[i.nr, v.id], GRB.LESS_EQUAL, v.Q, name='C4')
#     m.addConstr(quicksum( X[j.nr, depot_in.nr, v.id] for j in N), GRB.EQUAL, 1, name='C3**') 



#################################################### C5 & C6 ###############################################  
# Ensures that the total delivery load for a route is placed on the vehicle v, embarking on each trip, at the starting node itself. 
for v in V:

    # C5
    m.addConstr(D[0, v.id] - (quicksum(quicksum((X[i.nr, j.nr, v.id] * i.delivery_demand) for i in N) for j in T)), 
                GRB.EQUAL,  0, name='C5') 

for v in V:               
    # C6 
    m.addConstr(P[0, v.id], GRB.EQUAL, 0, name='C6')
 




#################################################### C7 & C8 ################################################
# Transit load constraints i.e., if arc (i, j) is visited by the vehicle v, 
# then the quantity to be delivered by the vehicle has to decrease by d_j while 
# the quantity picked-up has to increase by p_j
for v in V:
    for i in T: 
        for j in N: 
            # C7
            m.addConstr( (D[i.nr, v.id] - j.delivery_demand - D[j.nr, v.id]) * X[i.nr, j.nr, v.id], GRB.EQUAL, 0, name=f'C7_{i.nr}_{j.nr}_{v.id}')
for v in V:
    for i in T: 
        for j in N: 
            # C8
            m.addConstr( (P[i.nr, v.id] + j.pickup_demand - P[j.nr, v.id]) * X[i.nr, j.nr, v.id], GRB.EQUAL, 0, name=f'C8_{i.nr}_{j.nr}_{v.id}')


####################################################  C9  #####################################################
# Ensures the maximum route length for any vehicle v. 
for v in V:
    m.addConstr(quicksum(quicksum( distances[i.nr][j.nr] * X[i.nr, j.nr, v.id] for j in T) for i in T), GRB.LESS_EQUAL, v.TL, name='C9')




#################################################### C10 & C11 ###############################################
for v in V:
    for i in T:
        # C10
        m.addConstr(D[i.nr, v.id], GRB.GREATER_EQUAL, 0, name='C10')

        # C11
        m.addConstr(P[i.nr, v.id], GRB.GREATER_EQUAL, 0, name='C11')


CPU times: user 70.2 ms, sys: 3.79 ms, total: 74 ms
Wall time: 84.7 ms


# Optimization

In [95]:
m.write('MODEL.lp')
# Set time constraint for optimization (5minutes)
m.setParam('TimeLimit', 600)
# Set gap constraint for optimisation
# m.setParam('MIPgap', 0.05)

m.optimize()
m.write("solution.sol")
m.write("testout.sol")

status = m.status
if status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')

elif status == GRB.Status.OPTIMAL or True:
    f_objective = m.objVal
    print('***** RESULTS ******')
    print('\nObjective Function Value: \t %g' % f_objective)

elif status != GRB.Status.INF_OR_UNBD and status != GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)

Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-7267U CPU @ 3.10GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 135 rows, 448 columns and 2016 nonzeros
Model fingerprint: 0xfba15630
Model has 728 quadratic constraints
Variable types: 0 continuous, 448 integer (392 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+01, 4e+01]
  Objective range  [2e+01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve added 1290 rows and 0 columns
Presolve removed 0 rows and 30 columns
Presolve time: 0.04s
Presolved: 1425 rows, 418 columns, 5800 nonzeros
Variable types: 0 continuous, 418 integer (364 binary)

Root relaxation: objective 6.120000e+02, 104 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |    

# Verification

In [96]:
# Test function for checking feasibility:
def get_routes(V, T):
    route_lengths = {}
    routes = {}
    for v in V:
        # print(f'\nVehicle {v.id}:')
        routes[v.id] = []
        route_length = 0
        for i in T:
            for j in T:
                if X[i.nr, j.nr, v.id].X > 0.01:
                    # print(f'  {i.nr} --> {j.nr}') 
                    routes[v.id].append((i.nr, j.nr))
                    route_length += distances[i.nr][j.nr]
        route_lengths[v.id] = route_length

    return routes, route_lengths

def sort_routes(route_dict):
    """
    Sorts every route in the right order. Routes start and end at 0.
    
    :param route_dict: Dictionary containing routes with route IDs as keys
                       and lists of tuples (start, end) as values.
    :return: Dictionary with sorted routes.
    """
    sorted_routes = {}
    for vehicle, route in route_dict.items():
        # Start with an empty route
        sorted_route = []
        # Current location starts at 0 (depot)
        current_location = 0
        
        while route:
            # Find the next step in the route where the first element is the current location
            for i, (start, end) in enumerate(route):
                if start == current_location:
                    # Add this step to the sorted route
                    sorted_route.append((start, end))
                    # Remove this step from the route
                    route.pop(i)
                    # Update the current location to the end of this step
                    current_location = end
                    break
                    
        sorted_routes[vehicle] = sorted_route

    # # print routes
    # for vehicle, route in sorted_routes.items():
    #     # print(f'\nVehicle {vehicle}:')
    #     for (i, j) in route:
    #         if X[i, j, vehicle].X > 0.01:
    #             # print(f'  {i} --> {j}') 
    return sorted_routes

def print_routes(routes):
    for vehicle, route in routes.items():
        print(f'\nVehicle {vehicle}:')
        for (i, j) in route:
            if X[i, j, vehicle].X > 0.01:
                print(f'  {i} --> {j}') 

def double_visit(nodes, node):
    count = 0
    for number in nodes:
        if number == node:
            count += 1
    return True if count > 1 else False

def double_depart(nodes, node):
    count = 0
    for number in nodes:
        if number == node:
            count += 1
    return True if count > 1 else False

def customer_node_double_visit(routes, node):
    count = 0
    for vehicle, route in routes.items():
        for edge in route:
            visited = edge[1]
            if node == visited:
                count += 1
    return True if count > 1 else False

def delivery_and_pickup_demand(vehicle_id, route):
    '''returns the required total amounts to be deliverd and picked up in each route'''
    required_D = 0
    required_P = 0
    for edge in route:
        i = edge[0]
        j = edge[1]
        # print(f'From {i} to {j}')
        # print(f'Delivered = {demand["Delivery"].iloc[i]}')
        # print(f'Picked up = {demand["Pick-up"].iloc[i]}')
        required_D += demand["Delivery"].iloc[i]
        required_P += demand["Pick-up"].iloc[i]
    #     print(f' D_{i}, P_{i} = {required_D}, {required_P}')
    # print(f' D and P: {required_D}, {required_P}')    
    return required_D, required_P

def max_vehicle_load(vehicle_id, route):
    '''returns the maximum load a vehicle has carried in a certain route'''
    max_load = 0
    for edge in route:
        i = edge[0]
        j = edge[1]
        load = D[i, vehicle_id].X + P[i, vehicle_id].X
        j_pickup_demand = demand["Pick-up"].iloc[j]
        # print(f' \nEdge {edge}')
        # print(f' D_{i},{vehicle_id} = {D[i, vehicle_id].X}')
        # print(f' P_{i},{vehicle_id} = {P[i, vehicle_id].X}')
        # print(f'\nC8:')
        # print(f'(P_{i},{vehicle_id} + {j_pickup_demand} - P_{j},{vehicle_id}) * X_{i},{j},{vehicle_id} = 0')
        # print(f'({P[i, vehicle_id].X} + {j_pickup_demand} - {P[j, vehicle_id].X}) * {X[i, j, vehicle_id].X} = 0')
        if load > max_load:
            max_load = load

    return max_load 

def check_subtours_and_double_visits(routes):
    '''Check routes for subtours and double visits, return True or False for feasibility'''
    for vehicle, route in routes.items():
        nodes_to = [edge[1] for edge in route]
        nodes_from = [edge[0] for edge in route]
        # print(f'nodes_to {nodes_to} v={vehicle}')
        # print(f'nodes_from: {nodes_from} v={vehicle}')

        # Check if nodes are departed from more than once by same vehicle
        for node in nodes_from:
            if double_depart(nodes_from, node): 
                print(f'INFEASIBLE:\n Node {node} departed from more than once by vehicle {vehicle}.')
                return False

        # Check if nodes are visited more than once by same vehicle 
        for node in nodes_to:
            if double_visit(nodes_to, node):     
                print(f'INFEASIBLE:\nNode {node} visited more than once by Vehicle {vehicle}.')
                return False

        # Check if any customer nodes are visited more than once
        for customer_node in N:
            if customer_node_double_visit(routes, customer_node.nr):
                print(f'INFEASIBLE:\n Customer node {customer_node.nr} visited more than once.')
                return False
    return True


# Results

1. Routes
2. Route lengths
3. Max load
 

In [97]:
# Get results:
print(f'Objective Function value: {f_objective}')
routes, route_lengths = get_routes(V, T)

# Check for subtours and double visits:
if check_subtours_and_double_visits(routes) == True:
    # sort routes if the routes are contain no subtours or double visits of customer nodes
    routes = sort_routes(routes) 

# print routes:    
print_routes(routes)
print(f'Route lengths: {route_lengths}')


# Check if the maximum loads and route lengths do not exceed capacity
for vehicle, route in routes.items(): 
    v = next((v for v in V if v.id == vehicle), None)
    

    max_load = max_vehicle_load(vehicle, route)
    distance_travelled = route_lengths[vehicle]
    D_required, P_required = delivery_and_pickup_demand(vehicle, route)
    last_customer_node = route[-1][0]
    D_delivered, P_pickedup = D[0, vehicle].X, P[last_customer_node, vehicle].X
    


    # Test delivery and pickup quantities
    if abs(D_required - D_delivered) > 0.01:
        print(f'INFEASIBLE:\nDelivered quantities ({D_delivered}) by vehicle {vehicle} do not match demand ({D_required})')
        break
    
    if abs(P_required - P_pickedup) > 0.01:
        print(f'INFEASIBLE:\nPicked up quantities ({P_pickedup}) by vehicle {vehicle} do not match demand ({P_required})')
        break

    # Test max load
    if max_load > v.Q:
        print(f'INFEASIBLE:\n Vehicle {v.id} exceeded capacity')
        break

    # Test max distance
    if distance_travelled > v.TL:
        print(f'INFEASIBLE:\n Vehicle {v.id} exceeded Max Route Length')
        break

    print('\n')
    print(f'Vehicle {vehicle}:')
    print(f'    Max load carried by Vehicle {vehicle} is {max_load}')
    print(f'    Distance travelled by Vehicle {vehicle} is {route_lengths[vehicle]}')
    print(f'    Loads deliverd and picked-up: {D_delivered}, {P_pickedup}')
    print(f'    Demand for delivery and pickup: {D_required}, {P_required}')

Objective Function value: 911.0

Vehicle 1:
  0 --> 11
  11 --> 10
  10 --> 5
  5 --> 6
  6 --> 7
  7 --> 8
  8 --> 12
  12 --> 0

Vehicle 2:
  0 --> 9
  9 --> 4
  4 --> 3
  3 --> 2
  2 --> 1
  1 --> 13
  13 --> 0
Route lengths: {1: 416, 2: 495}


Vehicle 1:
    Max load carried by Vehicle 1 is 145.0
    Distance travelled by Vehicle 1 is 416
    Loads deliverd and picked-up: 145.0, 145.0
    Demand for delivery and pickup: 145, 145


Vehicle 2:
    Max load carried by Vehicle 2 is 145.0
    Distance travelled by Vehicle 2 is 495
    Loads deliverd and picked-up: 145.0, 145.0
    Demand for delivery and pickup: 145, 145
